In [7]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from numpy.linalg import det, inv
from math import sqrt, pi
import scipy.io
import matplotlib.pyplot as plt
import pickle
import sys

In [8]:
pickle_filename = "sample.pickle"
with open(pickle_filename, 'rb') as file:  
     X,y,Xtest,ytest = pickle.load(open('sample.pickle','rb'),encoding = 'latin1')

In [13]:
class_array = np.unique(y)
features_len = X.shape[1]
class_len = len(class_array)

indices = []
for i in range(class_len):
    indices.append(np.where(y==class_array[i])[0])

class_sum = []
for i in indices:
    total = 0;
    for j in i:
        total += X[j]
    class_sum.append(total)

lengths = []
for i in indices:
    lengths.append(len(i))


means = np.ones((class_len, features_len))
for i in range(X.shape[1]):
    for j in range(5):    
        means[j, i] = class_sum[j][i]/lengths[j]

#sub groups - global mean (mean corrected data):
globalMean= [np.mean(X[:, 0]), np.mean(X[:, 1])]

n = []
c_k = []
for i in range(class_len):
    n.append(len(indices[i])/len(X))
    mean_corrected_data = np.empty((len(indices[i]), features_len))
    for index, j in enumerate(indices[i]):
        mean_corrected_data[index] = X[j]-globalMean
    c_k.append(np.matmul(np.transpose(mean_corrected_data), mean_corrected_data)/ len(mean_corrected_data))

covmat = np.zeros((features_len, features_len))
for i in range(0, features_len):
    for j  in range(0, features_len):
        for k in range(class_len):
            covmat[i,j] += n[k]*c_k[k][i,j]


In [14]:
f = np.zeros((len(means), len(Xtest)))
for i in range(len(means)):
    f[i] = np.matmul(means[i], np.matmul(inv(covmat),np.transpose(Xtest)))- 0.5*np.matmul(means[i], np.matmul(inv(covmat), np.transpose(means[i])))
ypred = np.argmax(f, axis = 0) + np.ones((len(Xtest),))

#accuracy formula 
true_true = 0.0;
for i in range(len(ypred)):
    if(ypred[i] == ytest[i]):
        true_true += 1
acc = true_true/len(ypred) * 100

In [15]:
ypred

array([2., 2., 3., 4., 3., 2., 5., 4., 4., 4., 3., 5., 2., 2., 4., 1., 1.,
       1., 5., 3., 5., 1., 4., 3., 5., 1., 1., 2., 5., 2., 5., 5., 3., 5.,
       3., 4., 2., 3., 1., 2., 4., 5., 1., 1., 4., 4., 5., 3., 5., 5., 5.,
       4., 1., 5., 4., 4., 1., 3., 3., 1., 1., 4., 5., 3., 5., 4., 2., 3.,
       3., 3., 5., 2., 2., 1., 4., 4., 4., 1., 1., 4., 1., 3., 4., 5., 3.,
       4., 3., 3., 3., 2., 5., 1., 5., 2., 5., 4., 4., 5., 5., 1.])

In [16]:
acc

97.0